# Question 1 [25 Marks]

1. In this part, you are required to implement and evaluate the Apriori-based algorithm for frequent itemsets mining. **You are not allowed to use any libarary.** You are free to consult any online resources but be careful while implementing the solution. 

2. The program should be executable with 3 parameters: the name of the input dataset file, the threshold of minimum support count, and the name of the output file.  The minimum support count should be an integer.  An itemset is frequent if its support count is larger or equal to this threshold. 

3. There are 5 bonus points for storing the identified frequent itemsets in Excel file. 

4. The program should output a file that contains all the frequent itemsets together with their support.  The output file (sample output) should have the following format: each line contains a single frequent itemset as a list of items separated by whitespace.  At the end of the line, its support is printed between a pair of parenthesis.  For example: 1 2 3 (5) represents an itemset containing items 1, 2 and 3 with a support count of 5.


In [17]:
import pandas as pd
import numpy as np
import csv

In [2]:
#file reading from dataset.csv
#you are free to use any data loader i-e you can use pandas, np or any method you want.
df = pd.read_csv('dataset.csv', header=None, sep='\n')
df = df[0].str.split(",", expand = True) 
df.head()

,0,1,2,3,4,5,6,7,8,9,...,16,17,18,19,20,21,22,23,24,25
0,39,120,124,205,401,581,704,814,825,834,...,None,None,None,None,None,None,None,None,None,None
1,35,249,674,712,733,759,854,950,None,None,...,None,None,None,None,None,None,None,None,None,None
2,39,422,449,704,825,857,895,937,954,964,...,None,None,None,None,None,None,None,None,None,None
3,15,229,262,283,294,352,381,708,738,766,...,None,None,None,None,None,None,None,None,None,None
4,26,104,143,320,569,620,798,None,None,None,...,None,None,None,None,None,None,None,None,None,None


### Identify unique items [5 Marks]
From the list of transactions, identify and list all unique items in the dataset.

In [25]:
list1 = df.values.tolist()
uni = set()
for i in list1:
    for j in i:
        if j!=None:
            uni.add(j)
print(len(uni)) # 869
print(list(uni))

869
['629', '865', '68', '633', '683', '876', '328', '415', '514', '436', '595', '933', '920', '527', '731', '617', '673', '755', '13', '452', '969', '388', '743', '605', '82', '2', '671', '421', '189', '144', '862', '802', '319', '947', '733', '114', '593', '330', '461', '639', '150', '568', '451', '526', '3', '825', '245', '552', '583', '326', '329', '269', '55', '912', '164', '299', '81', '471', '8', '215', '500', '175', '660', '22', '638', '172', '966', '879', '335', '829', '120', '909', '874', '858', '25', '210', '161', '342', '152', '892', '278', '66', '214', '657', '42', '814', '632', '950', '188', '474', '886', '402', '831', '440', '396', '45', '489', '292', '870', '33', '544', '915', '872', '982', '127', '805', '418', '767', '551', '958', '385', '193', '499', '940', '162', '822', '74', '959', '653', '857', '348', '637', '849', '480', '501', '964', '439', '937', '923', '750', '773', '311', '355', '94', '524', '798', '603', '247', '453', '581', '565', '90', '513', '703', '749', 

## Implementation [20 Marks]

Test your implementation on the dataset dataset.csv and measure number of frequent itemsets with various minimum support values.  The test dataset is a synthetic dataset that contains approx 16,000 transactions.

In [23]:
#Create candidate 1 item sets
def createC1(dataSet):
    list1 = dataSet.values.tolist()
    C1 = {}
    for i in list1:
        for j in i:
            if j!=None:
                j_set = (j,)
                if j_set in C1.keys():
                    C1[j_set] += 1
                else:
                    C1[j_set] = 1
    return C1
C1 = createC1(df)
C1
# print(len(C1))

{('39',): 690,
 ('120',): 772,
 ('124',): 45,
 ('205',): 593,
 ('401',): 594,
 ('581',): 484,
 ('704',): 305,
 ('814',): 277,
 ('825',): 502,
 ('834',): 219,
 ('35',): 318,
 ('249',): 97,
 ('674',): 385,
 ('712',): 146,
 ('733',): 199,
 ('759',): 120,
 ('854',): 455,
 ('950',): 209,
 ('422',): 207,
 ('449',): 334,
 ('857',): 238,
 ('895',): 568,
 ('937',): 744,
 ('954',): 57,
 ('964',): 251,
 ('15',): 64,
 ('229',): 349,
 ('262',): 94,
 ('283',): 692,
 ('294',): 255,
 ('352',): 161,
 ('381',): 474,
 ('708',): 174,
 ('738',): 312,
 ('766',): 945,
 ('853',): 298,
 ('883',): 763,
 ('966',): 612,
 ('978',): 169,
 ('26',): 98,
 ('104',): 174,
 ('143',): 215,
 ('320',): 95,
 ('569',): 436,
 ('620',): 378,
 ('798',): 455,
 ('7',): 153,
 ('185',): 232,
 ('214',): 328,
 ('350',): 468,
 ('529',): 1144,
 ('658',): 300,
 ('682',): 718,
 ('782',): 450,
 ('809',): 325,
 ('849',): 79,
 ('947',): 608,
 ('970',): 290,
 ('979',): 22,
 ('227',): 291,
 ('390',): 409,
 ('71',): 585,
 ('192',): 296,
 ('208'

In [4]:
#Looking for k-frequent itemsets
def selectLk(dataSet,Ck,minSupport): #dataSet raw data set
    minSupportCount = int(len(dataSet)*minSupport + 0.5) # rounding so it's an int
#     print(minSupportCount)
    Lk = {}
    for key, val in Ck.items():
        if val >= minSupportCount:
            Lk.update({key:val})
    return Lk

Lk = selectLk(df,createC1(df), 0.02)
Lk
# len(Lk)

{('39',): 690,
 ('120',): 772,
 ('205',): 593,
 ('401',): 594,
 ('581',): 484,
 ('825',): 502,
 ('674',): 385,
 ('854',): 455,
 ('449',): 334,
 ('895',): 568,
 ('937',): 744,
 ('229',): 349,
 ('283',): 692,
 ('381',): 474,
 ('766',): 945,
 ('883',): 763,
 ('966',): 612,
 ('569',): 436,
 ('620',): 378,
 ('798',): 455,
 ('214',): 328,
 ('350',): 468,
 ('529',): 1144,
 ('682',): 718,
 ('782',): 450,
 ('809',): 325,
 ('947',): 608,
 ('390',): 409,
 ('71',): 585,
 ('279',): 478,
 ('280',): 333,
 ('597',): 436,
 ('675',): 474,
 ('720',): 625,
 ('914',): 631,
 ('183',): 647,
 ('217',): 906,
 ('276',): 427,
 ('653',): 414,
 ('161',): 370,
 ('175',): 445,
 ('177',): 731,
 ('571',): 466,
 ('795',): 543,
 ('960',): 400,
 ('392',): 378,
 ('862',): 576,
 ('27',): 366,
 ('78',): 420,
 ('775',): 618,
 ('921',): 403,
 ('411',): 327,
 ('579',): 364,
 ('778',): 425,
 ('803',): 378,
 ('523',): 376,
 ('614',): 457,
 ('888',): 576,
 ('944',): 443,
 ('70',): 383,
 ('204',): 367,
 ('334',): 328,
 ('480',): 3

In [6]:
def createCk(dataSet, Lk,k): # Lk: Frequent itemsets containing k items
    Ck = {}
    if k < 1:
        return Ck
    if k == 1:
        return createC1(dataSet)
    done = [] # list of sets
    for i in Lk:
        for j in Lk:
            if i!=j:
                if k == 2: # Lk is L1
                    if not {i, j} in done:
                        ke = i + j
                        Ck.update({ke: 0})
                        done.append({i, j})
                else:
                    set_i = set(i)
                    set_j = set(j)
                    common = set_i & set_j
                    if len(common) == k - 2 and not set_i | set_j in done:
                        ke = tuple(set_i | set_j)
                        Ck.update({ke: 0})
                        done.append(set_i | set_j)
    list1 = dataSet.values.tolist()
    for i in list1:
        i_set = set(i)
        for key, val in Ck.items():
            key_set = set(key)
            if key_set.issubset(i_set):
                Ck[key] += 1
    return Ck
Ck = createCk(df, selectLk(df,createC1(df), 0.02), 2)
Ck

{('39', '120'): 26,
 ('39', '205'): 15,
 ('39', '401'): 20,
 ('39', '581'): 14,
 ('39', '825'): 214,
 ('39', '674'): 42,
 ('39', '854'): 15,
 ('39', '449'): 5,
 ('39', '895'): 13,
 ('39', '937'): 32,
 ('39', '229'): 9,
 ('39', '283'): 53,
 ('39', '381'): 16,
 ('39', '766'): 28,
 ('39', '883'): 12,
 ('39', '966'): 21,
 ('39', '569'): 17,
 ('39', '620'): 12,
 ('39', '798'): 16,
 ('39', '214'): 12,
 ('39', '350'): 61,
 ('39', '529'): 45,
 ('39', '682'): 20,
 ('39', '782'): 16,
 ('39', '809'): 9,
 ('39', '947'): 24,
 ('39', '390'): 15,
 ('39', '71'): 20,
 ('39', '279'): 13,
 ('39', '280'): 13,
 ('39', '597'): 17,
 ('39', '675'): 15,
 ('39', '720'): 18,
 ('39', '914'): 16,
 ('39', '183'): 15,
 ('39', '217'): 30,
 ('39', '276'): 11,
 ('39', '653'): 16,
 ('39', '161'): 11,
 ('39', '175'): 10,
 ('39', '177'): 28,
 ('39', '571'): 15,
 ('39', '795'): 15,
 ('39', '960'): 14,
 ('39', '392'): 15,
 ('39', '862'): 17,
 ('39', '27'): 15,
 ('39', '78'): 11,
 ('39', '775'): 29,
 ('39', '921'): 10,
 ('39

In [12]:
#Apriori algorithm
def apriori(dataSet,minSupport):
    #create C1
    C1 = createC1(dataSet)
    
    #select L1
    L1 = selectLk(dataSet,C1,minSupport)
    
    #Iterative create and select Ck & Lk iteratively.
    done = False
    L = L1
    Lk = L1
    k = 2
    while not done:
        Ck = createCk(dataSet, Lk, k)
        Lk = selectLk(dataSet,Ck,minSupport)
        if len(Lk) == 0:
            done = True
        L.update(Lk)
        k += 1
    return L


In [13]:
# Do not change this cell
L = apriori(df ,0.02)
L

{('39',): 690,
 ('120',): 772,
 ('205',): 593,
 ('401',): 594,
 ('581',): 484,
 ('825',): 502,
 ('674',): 385,
 ('854',): 455,
 ('449',): 334,
 ('895',): 568,
 ('937',): 744,
 ('229',): 349,
 ('283',): 692,
 ('381',): 474,
 ('766',): 945,
 ('883',): 763,
 ('966',): 612,
 ('569',): 436,
 ('620',): 378,
 ('798',): 455,
 ('214',): 328,
 ('350',): 468,
 ('529',): 1144,
 ('682',): 718,
 ('782',): 450,
 ('809',): 325,
 ('947',): 608,
 ('390',): 409,
 ('71',): 585,
 ('279',): 478,
 ('280',): 333,
 ('597',): 436,
 ('675',): 474,
 ('720',): 625,
 ('914',): 631,
 ('183',): 647,
 ('217',): 906,
 ('276',): 427,
 ('653',): 414,
 ('161',): 370,
 ('175',): 445,
 ('177',): 731,
 ('571',): 466,
 ('795',): 543,
 ('960',): 400,
 ('392',): 378,
 ('862',): 576,
 ('27',): 366,
 ('78',): 420,
 ('775',): 618,
 ('921',): 403,
 ('411',): 327,
 ('579',): 364,
 ('778',): 425,
 ('803',): 378,
 ('523',): 376,
 ('614',): 457,
 ('888',): 576,
 ('944',): 443,
 ('70',): 383,
 ('204',): 367,
 ('334',): 328,
 ('480',): 3

In [14]:
# Bonus: Storing frequent itemsets in Excel file:
list_ = []
for key, val in L.items():
    row = ""
    for item in key:
        row += item + " "
    row += "("+str(val)+")"
    list_.append(row)
result1 = pd.Series(list_)
result1.to_excel("output1.xlsx") 

In [15]:
# Change the support count and report the effect of changing the minimum support count. Briefly explain the reason behind the change in resulting items.  
L = apriori(df, 0.01)
L

{('39',): 690,
 ('120',): 772,
 ('205',): 593,
 ('401',): 594,
 ('581',): 484,
 ('704',): 305,
 ('814',): 277,
 ('825',): 502,
 ('834',): 219,
 ('35',): 318,
 ('674',): 385,
 ('733',): 199,
 ('854',): 455,
 ('950',): 209,
 ('422',): 207,
 ('449',): 334,
 ('857',): 238,
 ('895',): 568,
 ('937',): 744,
 ('964',): 251,
 ('229',): 349,
 ('283',): 692,
 ('294',): 255,
 ('352',): 161,
 ('381',): 474,
 ('708',): 174,
 ('738',): 312,
 ('766',): 945,
 ('853',): 298,
 ('883',): 763,
 ('966',): 612,
 ('978',): 169,
 ('104',): 174,
 ('143',): 215,
 ('569',): 436,
 ('620',): 378,
 ('798',): 455,
 ('185',): 232,
 ('214',): 328,
 ('350',): 468,
 ('529',): 1144,
 ('658',): 300,
 ('682',): 718,
 ('782',): 450,
 ('809',): 325,
 ('947',): 608,
 ('970',): 290,
 ('227',): 291,
 ('390',): 409,
 ('71',): 585,
 ('192',): 296,
 ('208',): 223,
 ('279',): 478,
 ('280',): 333,
 ('496',): 240,
 ('530',): 188,
 ('597',): 436,
 ('618',): 203,
 ('675',): 474,
 ('720',): 625,
 ('855',): 161,
 ('914',): 631,
 ('932',):

Reason:
When the minimum support was 0.02, the minimum support count was around 320. At this point, all frequent itemsets were of 
length 1 since there did exist a single 2-itemset with a minimum support count of 320 or more. By reducing minimum support by half, the count became 160 so now a lot more 1-itemsets showed up as well as 2-itemsets and a 3-itemset.

In [16]:
# Bonus: Storing frequent itemsets in Excel file:
list_ = []
for key, val in L.items():
    row = ""
    for item in key:
        row += item + " "
    row += "("+str(val)+")"
    list_.append(row)
result2 = pd.Series(list_)
result2.to_excel("output2.xlsx") 